### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [155]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [156]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [157]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [158]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis"

Path(path).mkdir(parents=True, exist_ok=True)

In [159]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [160]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [161]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [162]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [163]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Talk Time/Data/\
1-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s
...,...,...,...,...,...,...
119043,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
119044,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
119045,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
119046,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...


In [164]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s,"[0, 60, 33, 12]"
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s,"[0, 33, 29, 136]"
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...,"[33, 4, 33, 3]"
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...,"[0, 60, 33, 3]"
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s,"[8, 3, 67, 40]"
...,...,...,...,...,...,...,...
119043,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
119044,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
119045,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
119046,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."


In [165]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [166]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s,"[0, 60, 33, 12]"
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s,"[0, 33, 29, 136]"
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...,"[33, 4, 33, 3]"
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...,"[0, 60, 33, 3]"
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s,"[8, 3, 67, 40]"
...,...,...,...,...,...,...,...
65970,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
65971,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
65972,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
65973,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."


In [167]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [168]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [169]:
#word_num_index_dict

In [170]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [171]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[128,
 138,
 139,
 140,
 144,
 151,
 154,
 26,
 155,
 28,
 157,
 160,
 163,
 165,
 166,
 169,
 170,
 171,
 175,
 176,
 52,
 180,
 184,
 185,
 187,
 189,
 62,
 190,
 193,
 194,
 196,
 199,
 74,
 80,
 108,
 114,
 122,
 126,
 127]

In [172]:
#Counter(index_list)

In [173]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s,"[0, 60, 33, 12]"
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s,"[0, 33, 29, 136]"
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...,"[33, 4, 33, 3]"
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...,"[0, 60, 33, 3]"
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s,"[8, 3, 67, 40]"
...,...,...,...,...,...,...,...
182,hebben mensen echt niemand,211,214,soms hebben mensen echt niemand,3Nj4Kvd5zAI,https://www.youtube.com/watch?v=3Nj4Kvd5zAI&t=...,"[49, 115, 92, 177]"
183,komen iedereen wordt gewoon,4,6,komen iedereen wordt gewoon onmiddellijk een n...,S3RFzHMQdsg,https://www.youtube.com/watch?v=S3RFzHMQdsg&t=4s,"[118, 172, 135, 121]"
184,hebben moeten doen uit onze,86,89,dat we constant grepen hebben moeten doen uit ...,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=86s,"[49, 83, 58, 59, 112]"
185,zoals jullie hebben kunnen zien,286,288,zoals jullie hebben kunnen zien,WFoMFhhgDEg,https://www.youtube.com/watch?v=WFoMFhhgDEg&t=...,"[162, 69, 49, 84, 95]"


In [174]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s,"[0, 60, 33, 12]"
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s,"[0, 33, 29, 136]"
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...,"[33, 4, 33, 3]"
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...,"[0, 60, 33, 3]"
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s,"[8, 3, 67, 40]"
...,...,...,...,...,...,...,...
182,hebben mensen echt niemand,211,214,soms hebben mensen echt niemand,3Nj4Kvd5zAI,https://www.youtube.com/watch?v=3Nj4Kvd5zAI&t=...,"[49, 115, 92, 177]"
183,komen iedereen wordt gewoon,4,6,komen iedereen wordt gewoon onmiddellijk een n...,S3RFzHMQdsg,https://www.youtube.com/watch?v=S3RFzHMQdsg&t=4s,"[118, 172, 135, 121]"
184,hebben moeten doen uit onze,86,89,dat we constant grepen hebben moeten doen uit ...,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=86s,"[49, 83, 58, 59, 112]"
185,zoals jullie hebben kunnen zien,286,288,zoals jullie hebben kunnen zien,WFoMFhhgDEg,https://www.youtube.com/watch?v=WFoMFhhgDEg&t=...,"[162, 69, 49, 84, 95]"


In [175]:
df_youtube_sent_sample.drop("word_index", axis=1, inplace=True)

In [176]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [177]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s
...,...,...,...,...,...,...
182,hebben mensen echt niemand,211,214,soms hebben mensen echt niemand,3Nj4Kvd5zAI,https://www.youtube.com/watch?v=3Nj4Kvd5zAI&t=...
183,komen iedereen wordt gewoon,4,6,komen iedereen wordt gewoon onmiddellijk een n...,S3RFzHMQdsg,https://www.youtube.com/watch?v=S3RFzHMQdsg&t=4s
184,hebben moeten doen uit onze,86,89,dat we constant grepen hebben moeten doen uit ...,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=86s
185,zoals jullie hebben kunnen zien,286,288,zoals jullie hebben kunnen zien,WFoMFhhgDEg,https://www.youtube.com/watch?v=WFoMFhhgDEg&t=...


In [178]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [179]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,aan,7
1,nou,7
2,niet,7
3,net,7
4,nee,7
...,...,...
156,denk,1
157,tijd,1
158,twee,1
159,vinden,1


In [180]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [181]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [182]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [183]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [184]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['hou',
 'omdat',
 'z',
 'h',
 'altijd',
 'vader',
 'doet',
 'n',
 'wacht',
 'ben',
 'dacht',
 'geef',
 'vrouw',
 'andere',
 'dank',
 'mr',
 'bedankt',
 'wilt',
 'waren',
 'spijt',
 'haar',
 'praten',
 'niks',
 'gezien',
 'zitten',
 'moeder',
 'bent',
 'zullen',
 'helpen',
 'vind',
 'wist',
 'neem',
 'hebt',
 'waarom',
 'iemand',
 'nodig',
 'leven',
 'zeggen',
 'tegen']

In [185]:
set_not_in_sent_word = set(not_in_sent_word)

In [186]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [187]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'altijd',
 'andere',
 'bedankt',
 'ben',
 'bent',
 'dacht',
 'dank',
 'doet',
 'geef',
 'gezien',
 'h',
 'haar',
 'hebt',
 'helpen',
 'hou',
 'iemand',
 'leven',
 'moeder',
 'mr',
 'n',
 'neem',
 'niks',
 'nodig',
 'omdat',
 'praten',
 'spijt',
 'tegen',
 'vader',
 'vind',
 'vrouw',
 'waarom',
 'wacht',
 'waren',
 'wilt',
 'wist',
 'z',
 'zeggen',
 'zitten',
 'zullen'}

In [188]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['bent',
 'haar',
 'zitten',
 'z',
 'omdat',
 'vader',
 'waarom',
 'vrouw',
 'doet',
 'waren',
 'wacht',
 'n',
 'wist',
 'tegen',
 'hebt',
 'bedankt',
 'geef',
 'zullen',
 'dank',
 'gezien',
 'niks',
 'ben',
 'altijd',
 'spijt',
 'wilt',
 'moeder',
 'neem',
 'andere',
 'dacht',
 'h',
 'iemand',
 'mr',
 'nodig',
 'praten',
 'vind',
 'helpen',
 'zeggen',
 'leven',
 'hou']

In [189]:
len(not_in_sent_selected_word_list)

39

In [190]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...
4,en de al zo,6,7,en die komt bovenop de lockdown en de al zo in...,x8scbsqJBJE,https://www.youtube.com/watch?v=x8scbsqJBJE&t=6s
...,...,...,...,...,...,...
65970,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
65971,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...
65972,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
65973,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...


In [191]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [192]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [193]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,bent,u bent op de,32,34,u bent op de bestemming,A8EXbpOKPGk,https://www.youtube.com/watch?v=A8EXbpOKPGk&t=32s
1,bent,bent u zo in,89,91,vanuit ossendrecht bent u zo in bergen op zoom...,VGYeODPmTg8,https://www.youtube.com/watch?v=VGYeODPmTg8&t=89s
2,bent,ja je bent er,134,135,ja je bent er zelf ook een beetje mee opgegroeid,iVMNi3yMNuo,https://www.youtube.com/watch?v=iVMNi3yMNuo&t=...
3,bent,u bent nu bij,173,177,heer u bent nu bij god hij gaf u de macht,HlNj8wx9ZUk,https://www.youtube.com/watch?v=HlNj8wx9ZUk&t=...
4,bent,bent u op een,164,165,bent u op een plek waar u een coronatestbewijs...,96qWsXqcQUo,https://www.youtube.com/watch?v=96qWsXqcQUo&t=...
...,...,...,...,...,...,...,...
182,hou,en ik hou me,91,93,mijn naam is hans en ik hou me bezig met ontkn...,YqBtp_VcJZY,https://www.youtube.com/watch?v=YqBtp_VcJZY&t=91s
183,hou,hou je je aan,181,183,hou je je aan alle coronamaatregelen,U6LuV85Elcw,https://www.youtube.com/watch?v=U6LuV85Elcw&t=...
184,hou,ik hou van de,234,236,ik sta gecorrigeerd ik hou van de regio vlaams...,wFrPC1GDEd4,https://www.youtube.com/watch?v=wFrPC1GDEd4&t=...
185,hou,ik hou van de,53,55,ik hou van de manier waarop ze is,pBns1NireyI,https://www.youtube.com/watch?v=pBns1NireyI&t=53s


In [194]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [195]:
not_in_sent_selected_word_list_result.word.nunique()

39

In [196]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [197]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [198]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [199]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,bent,u bent op de,32,34,u bent op de bestemming,A8EXbpOKPGk,https://www.youtube.com/watch?v=A8EXbpOKPGk&t=32s
1,bent,bent u zo in,89,91,vanuit ossendrecht bent u zo in bergen op zoom...,VGYeODPmTg8,https://www.youtube.com/watch?v=VGYeODPmTg8&t=89s
2,bent,ja je bent er,134,135,ja je bent er zelf ook een beetje mee opgegroeid,iVMNi3yMNuo,https://www.youtube.com/watch?v=iVMNi3yMNuo&t=...
3,bent,u bent nu bij,173,177,heer u bent nu bij god hij gaf u de macht,HlNj8wx9ZUk,https://www.youtube.com/watch?v=HlNj8wx9ZUk&t=...
4,bent,bent u op een,164,165,bent u op een plek waar u een coronatestbewijs...,96qWsXqcQUo,https://www.youtube.com/watch?v=96qWsXqcQUo&t=...
...,...,...,...,...,...,...,...
182,hou,en ik hou me,91,93,mijn naam is hans en ik hou me bezig met ontkn...,YqBtp_VcJZY,https://www.youtube.com/watch?v=YqBtp_VcJZY&t=91s
183,hou,hou je je aan,181,183,hou je je aan alle coronamaatregelen,U6LuV85Elcw,https://www.youtube.com/watch?v=U6LuV85Elcw&t=...
184,hou,ik hou van de,234,236,ik sta gecorrigeerd ik hou van de regio vlaams...,wFrPC1GDEd4,https://www.youtube.com/watch?v=wFrPC1GDEd4&t=...
185,hou,ik hou van de,53,55,ik hou van de manier waarop ze is,pBns1NireyI,https://www.youtube.com/watch?v=pBns1NireyI&t=53s


In [200]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [201]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx']

In [202]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [203]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass